# Alongshore transport

## First import some necessary packages

In [3]:
import logging
import pathlib
import sys
import warnings

import colorcet as cc
import dotenv
import geopandas as gpd
import holoviews as hv
import hvplot.pandas  # noqa: API import
import numpy as np
import pandas as pd
import panel as pn
import pooch

from bokeh.models import PanTool, WheelZoomTool, HoverTool
from bokeh.resources import INLINE
import bokeh.io

import coastal_dynamics as cd

# Activate Panel extension to make interactive visualizations
pn.extension()

In [ ]:
# # Read questions from cloud storage
questions = cd.read_questions(
    "./7_alongshore_transport.json"
    # "az://coastal-dynamics/questions/5_morphodynamics_upper_shoreface.json",
    # storage_options={"account_name": "coclico"},
)

# Alongshore sediment transport

Welcome to the notebook of week 6! The main topic of this notebook is cross-shore sediment transport. This notebook is relatively short. We will look into the CERC formula, and how to use it to obtain an (S, $\phi$-curve. Afterwards, we will use the (S, $\phi$-curve to determine coastal evolution around a breakwater.

## CERC formula and ($S$,$\phi$)-curve
Many different formulas exist to calculate bulk longshore sediment transport. One widely used formula is the CERC formula (Equation 8.4 in the book). We will use a version here which uses deep-water parameters. This formula, which is applicable for straight, parallel depth contours, is defined as:
$$S = \frac{K}{32(s-1)(1-p)} c_b \sin{2 \phi_0} H_0^2 $$
For a complete description of each parameter, see section 8.2.3 of the book.

For simplicity, we will assume that the offshore wave height $H_0$ is equal to the deep-water root-mean-square wave height $H_{rms,0}$, and can be represented by a single value of 1.8 m, with a period T of 6.5 s. This gives a value for $K$ of 0.7.

You might notice that the only variables left in the CERC formula are the breaking wave celerity $c_b$ and the offshore wave angle $\phi_0$. Starting from our values for $H_0$ and $T$, we can infer values for $c_b$ using linear wave theory with the dispersion relation. The function below does exactly that. Each step in the function is explained. Note that the value for $c_b$ depends on the angle of incidence.

In [ ]:
# Maybe we don't distribute this function, and instead keep it private :-)

def disper(w, h, g=9.8):
    '''
    DISPER  Linear dispersion relation.
    
    absolute error in k*h < 5.0e-16 for all k*h
    
    Syntax:
    k = disper(w, h, [g])
    
    Input:
    w = 2*pi/T, were T is wave period
    h = water depth
    g = gravity constant
    
    Output:
    k = wave number
    
    Example
    k = disper(2*pi/5,5,g = 9.81);
    
    Copyright notice
    --------------------------------------------------------------------
    Copyright (C) 
    G. Klopman, Delft Hydraulics, 6 Dec 1994
    M. van der Lugt conversion to python, 11 Jan 2021
    
    '''    
    #make sure numpy array
    listType = type([1,2])
    Type = type(w)

    w = np.atleast_1d(w)
    
    #check to see if warning disappears
    wNul = w==0
    w[w==0] = np.nan

    
    w2 = w**2*h/g
    q = w2 / (1-np.exp(-w2**(5/4)))**(2/5)
    
    for j in np.arange(0,2):
        thq = np.tanh(q)
        thq2 = 1-thq**2
        aa = (1 - q*thq) * thq2
        
        #prevent warnings, we don't apply aa<0 anyway
        aa[aa<0] = 0
        
        bb = thq + q*thq2
        cc = q*thq - w2
        
        
        D = bb**2-4*aa*cc
        
        # initialize argument with the exception
        arg = -cc/bb
        
        # only execute operation on those entries where no division by 0 
        ix = np.abs(aa*cc)>=1e-8*bb**2 
        arg[ix] = (-bb[ix]+np.sqrt(D[ix]))/(2*aa[ix]) 

                
        q = q + arg

              
    k = np.sign(w)*q/h
    
    #set 0 back to 0
    k = np.where(wNul,0,k)

    #if input was a list, return also as list
    if Type==listType:
        k = list(k)
    elif len(k)==1:
        k = k[0]
        
    return k

In [26]:
def find_cb(phi0, H0, T, 
            g=9.81, gamma=0.8, hb_guess=np.arange(0.1, 5.0, 0.01),
            print_report=False):
    """
    Returns breaking wave celerity cb [m/s] for given:
    - phi0 : angle of incidence [degrees]
    - H0   : deep water wave height [m]
    - T    : period [s]

    The parameter hb_guess is used as guessed values for the breaking depth. 
    From this array, the best-fitting value is chosen in the end. You can adjust this
    array to make estimates more accurate at the cost of computational efficiency. 
    """
    # First convert the angle of incidence to radians
    phi_rad = phi0 / 360 * 2 * np.pi
    
    # We start with calculating deep water celerity, wavelength, and angular frequency
    c0 = g * T / (2 * np.pi)
    L0 = c0 * T
    w  = T / (2 * np.pi)

    # For every value of hb_guess, the wavenumber k is determined using the dispersion relation
    k = disper(w, hb, g=g)  # Feel free to use your own implementation from week 2!

    # Next we calculate the celerity and group celerity for each breaking depth
    c = np.sqrt(g / k * np.tanh(k * hb))
    n = 1/2 * (1 + (2 * k * hb) / (np.sinh(2 * k * hb)))
    cg = n * c

    # In order to correctly shoal the waves, we also need the deep water group celerity
    n0 = 1/2
    cg0 = n0 * c0

    # And to account for refraction we need the angle of incidence at breaking
    phi = np.arcsin(np.sin(phi_rad) / c0 * c)
    
    # Shoaling & refraction coefficients
    Ksh = np.sqrt(cg0 / cg)
    Kref = np.sqrt(np.cos(phi_rad)/np.cos(phi))

    # Wave heights Hb at depth hb
    Hb = Ksh * Kref * H0

    # We are looking for an hb where the breaker parameter is 0.8
    # We can determine which value of hb in our array gets closest using the
    # following line of code:
    i = np.argmin(np.abs(Hb / hb - gamma))
    Hb_pred, hb_pred = Hb[i], hb[i]

    # Let's print what we found
    if print_report:
        print(f'predicted breaking depth: {hb_pred:.2f}')
        print(f'predicted breaking wave height: {Hb_pred:.2f}')
        print(f'gamma = {Hb_pred / hb_pred:.2f}')

    # And finally return the associated value for cb
    return c[i]

Let's check that it works. The cell below shows that for an offshore wave height of 2 m, a period of 7 s, and a angle of incidence of 5 degrees, we get a $c_b$ of 4.92 m/s. You can check that this is in reasonable agreement with Example 8.1 in the book.

In [29]:
phi0 = 5
H0 = 2
T = 7

print(f'cb: {find_cb(phi0, H0, T, print_report=True):.2f}')

predicted breaking depth: 2.79
predicted breaking wave height: 2.23
gamma = 0.80
cb: 4.92


Now that we have this function, we can calculate a breaking wave celerity for each angle of incidence.

In [32]:
# Set the range for which we want to calculate cb
phi_array = np.arange(-80, 80, 1)

# Initialize cb array
cb_array  = np.zeros(phi_array.shape)

# Loop through each phi and compute associated value for cb
for i in range(len(phi_array)):
    cb_array[i] = find_cb(phi_array[i], H0, T)

With cb, we can calculate our transport $S$ as a function of $\phi$, which means we can generate an ($S$,$\phi$)-curve! 

Remember that we can use the CERC formulation of Equation 8.10 for this. We use typical values of $K=0.7$, $p=0.4$, and $s=2.65$. 

Finish the equation below to compute the bulk sediment transport S.

In [34]:
def CERC(cb, phi0, H0,
         K=0.7,
         s=2.65,
         p=0.4
        ):
    
    S = K / (32 * (s - 1) * (1 - p)) * cb * np.sin(2*(phi0 / 360 * 2 * np.pi)) * H0**2

    return S

In [35]:
S = CERC(cb_array, phi_array, H0)

In [47]:
warnings.filterwarnings("ignore")
logging.getLogger().setLevel(logging.ERROR)

(hv.Curve((phi_array, S)) * hv.HLine(0).opts(color='black') * hv.VLine(0).opts(color='black')).opts(xlabel='angle [degrees]', ylabel='S [m3/s]', title='(S, phi)-curve', width=800, height=400, show_grid=True)

:Overlay
   .Curve.I :Curve   [x]   (y)
   .HLine.I :HLine   [x,y]
   .VLine.I :VLine   [x,y]

This is the end of the first part of this notebook.

## Coastal evolution around a breakwater
We will now use the ($S$,$\phi$)-curve generated above to predict coastal evolution around a breakwater. Firstly, we need a wave climate. We use a simplified version of Table 8.2 from the book, with only two wave conditions:
* Condition 1: $H_0=2$ m with $T=8$ s and $\phi_0=40$ degrees. This condition occurs 35% of the time.
* Condition 2: $H_0=1.2$ m with $T=12$ and $\phi_0=-25$ degrees. This condition occurs 50% of the time.

15% of the time, no significant waves are present. Remember that positive angles result in positive transport (see also the image in Table 8.2).

Let's determine the average yearly transport for these conditions. For both conditions, we can determine the transport $S$.

In [57]:
# For condition 1
cb1 = find_cb(40, 2, 8)
S1  = CERC(cb1, 40, 2)

# For condition 2
cb2 = find_cb(-25, 1.2, 12)
S2  = CERC(cb2, -25, 1.2)

# These transports S are both given in m3/s. Remember that condition 1 occurs 35% of the year, and condition 2 occurs 50% of the year. For 1 year, the contributions are:
S_total = (0.35 * S1 + 0.5 * S2) * 365.25 * 24 * 3600

print(f'Total yearly transport: {S_total:.0f} m3/year')

Total yearly transport: 2997039 m3/year


This total transport is positive, which means the cumulative transport is in positive direction.